<a href="https://colab.research.google.com/github/bikaneal/ml_thesis/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Применение алгоритма классификации на репликах участников групповых работ**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# С использованием логистической регрессии и мешка слов

Более простой вариант

In [ ]:
!pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from collections import Counter

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Загружаем и обрабатываем датасет



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Classes/file_upd.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
df

,text,category
0,Давайте двойку.,предложение
1,Ставлю.,предложение
2,Нам надо пять поставить.,предложение
3,"Ребят, 2 надо ставить.",предложение
4,"2 надо ставить, у нас ресурсов нет.",предложение
...,...,...
260,То есть у нас сейчас получается будет два раун...,другое
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое
263,Давай,другое


In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
morph = pymorphy3.MorphAnalyzer()
stop_words = stopwords.words('russian')
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»']

In [ ]:
df['preprocessed_text'] = df.apply(lambda row: preprocess(row['text'], stop_words, punctuation_marks, morph), axis=1)

In [ ]:
df

,text,category,preprocessed_text
0,Давайте двойку.,предложение,"[давать, двойка]"
1,Ставлю.,предложение,[ставить]
2,Нам надо пять поставить.,предложение,"[пять, поставить]"
3,"Ребят, 2 надо ставить.",предложение,"[ребята, 2, ставить]"
4,"2 надо ставить, у нас ресурсов нет.",предложение,"[2, ставить, ресурс]"
...,...,...,...
260,То есть у нас сейчас получается будет два раун...,другое,"[получаться, раунд]"
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое,"[смотреть, наблюдать, стратегия]"
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое,"[варвар, каждый, отправить, ещё, ответ, следую..."
263,Давай,другое,[давать]


### Подсчет частоты слов во всех репликах

In [ ]:
words = Counter()

In [ ]:
for phrase in df['preprocessed_text']:
  words.update(phrase)

In [ ]:
words.most_common(10)

[('давать', 49),
 ('поставить', 43),
 ('ставить', 28),
 ('мочь', 27),
 ('остаться', 20),
 ('всё', 20),
 ('команда', 19),
 ('это', 18),
 ('выиграть', 12),
 ('просто', 12)]

###Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:

* Код заполнитель: 0

* Неизвестное слово: 1

Нумерация в словаре начинается с 2.

In [ ]:
# Словарь, отображающий слова в коды
word_to_index = dict()

# Словарь, отображающий коды в слова
index_to_word = dict()

###Заполняем константы:

In [ ]:
max_words = len(words)
random_state = 42

###Создаем словари

In [ ]:
for i, word in enumerate(words.most_common(max_words - 2)):
  word_to_index[word[0]] = i+2
  index_to_word[i+2] = word[0]

###Функция для преобразования список слов в список кодов

In [ ]:
def text_to_sequence(txt, word_to_index):
  seq = []
  for word in txt:
    index = word_to_index.get(word, 1)
    if index != 1:
      seq.append(index)
  return seq

###Преобразуем все реплики в последовательность кодов

In [ ]:
df['sequences'] = df.apply(lambda row: text_to_sequence(row['preprocessed_text'], word_to_index), axis=1)

In [ ]:
df

,text,category,preprocessed_text,sequences
0,Давайте двойку.,предложение,"[давать, двойка]","[2, 32]"
1,Ставлю.,предложение,[ставить],[4]
2,Нам надо пять поставить.,предложение,"[пять, поставить]","[44, 3]"
3,"Ребят, 2 надо ставить.",предложение,"[ребята, 2, ставить]","[114, 26, 4]"
4,"2 надо ставить, у нас ресурсов нет.",предложение,"[2, ставить, ресурс]","[26, 4, 27]"
...,...,...,...,...
260,То есть у нас сейчас получается будет два раун...,другое,"[получаться, раунд]","[28, 56]"
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое,"[смотреть, наблюдать, стратегия]","[12, 409, 33]"
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое,"[варвар, каждый, отправить, ещё, ответ, следую...","[197, 78, 16, 42]"
263,Давай,другое,[давать],[2]


## Готовим данные для обучения

###Преобразуем текстовые метрки данных в числовые

In [ ]:
mapping = {'предложение': 1, 'другое': 0}

In [ ]:
df.replace({'category': mapping}, inplace=True)

In [ ]:
df

,text,category,preprocessed_text,sequences
0,Давайте двойку.,1,"[давать, двойка]","[2, 32]"
1,Ставлю.,1,[ставить],[4]
2,Нам надо пять поставить.,1,"[пять, поставить]","[44, 3]"
3,"Ребят, 2 надо ставить.",1,"[ребята, 2, ставить]","[114, 26, 4]"
4,"2 надо ставить, у нас ресурсов нет.",1,"[2, ставить, ресурс]","[26, 4, 27]"
...,...,...,...,...
260,То есть у нас сейчас получается будет два раун...,0,"[получаться, раунд]","[28, 56]"
261,"Смотрим их, да, и наблюдаем за стратегией других.",0,"[смотреть, наблюдать, стратегия]","[12, 409, 33]"
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",0,"[варвар, каждый, отправить, ещё, ответ, следую...","[197, 78, 16, 42]"
263,Давай,0,[давать],[2]


###Разделяем метки классов и данные для обучения

In [ ]:
x_train_seq, y_train = df['sequences'], df['category']

###Создаем мешок слов

In [ ]:
def vectorize_sequences(sequences, dimension=len(words)):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    for index in sequence:
      results[i, index] += 1.
  return results

In [ ]:
x_vectors = vectorize_sequences(x_train_seq, max_words)

In [ ]:
x_vectors.shape

(265, 410)

## Создаем модель машинного обучения

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

# Инициализация модели логистической регрессии
lr = LogisticRegression(random_state = random_state)

# Создание скорера для f1-score
f1_scorer = make_scorer(f1_score)

# Выполнение кросс-валидации с вычислением f1-score
f1_scores = cross_val_score(lr, x_vectors, y_train, cv=5, scoring=f1_scorer)

# Вывод результатов
print("Средний F1-счет: %0.2f (+/- %0.2f)" % (f1_scores.mean(), f1_scores.std() * 2))


Средний F1-счет: 0.66 (+/- 0.24)


###Обучаем модель

Для дальнейших предсказаний

In [ ]:
lr.fit(x_vectors, y_train)

LogisticRegression(random_state=42)

## Экспериментальное внедрение модели

In [ ]:
speakers = dict()

In [ ]:
test = ['Говорящий 1: Всем привет. Мы, кажется, все собрались, да? У нас должно было быть четверо. Меня хорошо слышно? Хорошо. Я, простите, больше не буду без камеры. У нас сейчас важный такт. Нам нужно выбрать, кто из нас будет подавать заявки в форме. И я пришлю форму.',
        'Говорящий 2: Я буду делать.',
        'Говорящий 1: Отлично, давайте. Сейчас вам в чате зума. Все, получили? А сколько у нас минут заполнено?',
        'Говорящий 2: Пять минут.',
        'Говорящий 1: Ага. Тогда какие предложения по ставкам?',
        'Говорящий 2: Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
        'Говорящий 1: Ну да, мне тоже кажется, что или надо больше половины, чтобы сразу на свою сторону, или минимальную, чтобы другая команда ресурсы пожгла.',
        'Говорящий 2: Угу. Так. Проверочный код вашей команды. А где его проверочный код получить?',
        'Говорящий 1: А вот тоже я отправила сообщение тут же.',
        'Говорящий 1: А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
        'Говорящий 2: Ну 13, да, интересно в принципе ставка, если с учетом того, что она заканчивается на неудобную цифру. Так, а у нас играет кто получается?',
        'Говорящий 1: Мы наверное с четвертой играем.',
        'Говорящий 2: У нас осталось 1 минута. Так, давайте, кто за единицу голосует.',
        'Говорящий 1: Я за единицу. Ага.',
        'Говорящий 2: Я за единицу. И... Я за единицу.',
        'Говорящий 2: Так, отлично. Так, единица или 13?',
        'Говорящий 1: Галина, я вас плохо слышу, если честно.',
        'Говорящий 2: Так, а мы разговариваем, Милана, с вами, да, вдвоем?',
        'Говорящий 1: Я так понимаю, у Анны что-то с микрофоном? Юлия, вы с нами?',
        'Говорящий 2: Давайте, все, отправляю единицу. Ставку сделали. Нужно выйти, да, теперь?',
        'Говорящий 1: Она сама отправится.',
        'Говорящий 2: А у меня пришло приглашение выйти из зала.',
        'Говорящий 1: О, отлично. Спасибо, Юлия. Галина, вы с нами? Все, мы все теперь в чате. Хорошо.',
        'Говорящий 1: Что мы тогда ставим, 21 или другие варианты есть?',
        'Говорящий 1: Так, Юлия Анна пишет 21, Галина 29.',
        'Говорящий 1: Нет, друзья, у них максимум 20 ставка, нам же не нужно 29. У нас 21 это максимум.',
        'Говорящий 1: Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
        'Говорящий 1: Да, точно. У всех, видимо, что-то со звуком. Ничего, я буду озвучивать.',
        'Говорящий 2: Так, все здесь?',
        'Говорящий 1: Здесь.',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 1: У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
        'Говорящий 2: Ну, смотрите, а это же будет последний ход?',
        'Говорящий 1: Что?',
        'Говорящий 1: Это же будет последний раунд?',
        'Говорящий 1: Нет, мы играем пока не перетащим камень или пока не закончатся монеты.',
        'Говорящий 2: Но я предлагаю 7. Почему? Потому что даже если они поставят чуть больше 8, у них в остатке останется 11, а у нас тут чуть больше монет 21.',
        'Говорящий 1: Ну, здесь я просто не знаю, насколько велика вероятность, что они поставят меньше 7.',
        'Говорящий 2: Нет, ну тут меньше 7 они точно не поставят. Хотя, с другой стороны.',
        'Говорящий 1: Мы можем поставить чуть больше половины. Просто тогда будет вероятность больше, что мы...',
        'Говорящий 2: Если они перетянут, допустим, если они поставят 8, и  них в остатке останется 11, а у нас 21, то у нас будет преимущество на следующие шаги, у нас останется чуть больше монет. Просто если они сейчас поставят 8, мы поставим 10, и мы перетянем камень, у нас останется 18.',
        'Говорящий 1: Поздравляю, в одном шаге от победы.',
        'Говорящий 2: Так, я предлагаю поставить 7.',
        'Говорящий 1: Так мы можем все поставить, мы уже точно их выиграем, если поставим 18. У них меньше. Так что ставим 18 и все. Камень наш.',
        'Говорящий 2: А, ну да, логично. Так, а почему-то у меня не возвращается моя ставка.',
        'Говорящий 1: 18 у нас же, правильно?',
        'Говорящий 2: Да, я ставлю 18, конечно, заканчиваем.',
        'Говорящий 1: Супер.',
        'Говорящий 2: 18, правильно.',
        'Говорящий 1: Интересно, если бы мы поставили 15, мы бы вышли с монетами?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Ну, очень, как будто слабо. Сейчас слышно.',
        'Говорящий 2: А что нам делать-то нужно?',
        'Говорящий 1: Пока мы отдыхаем. И ждем следующий раунд.',
        'Говорящий 2: Я поняла.',
        'Говорящий 1: Можем, не знаю, подумать, что... Может быть, в следующий раунд, короче, на следующем ходе, можно зафиксировать, какие стратегии команды-победители используют, чтобы примерно знать, может быть, с кем может столкнуться в следующем раунде. Посмотреть пока будет демонстрировать.',
        'Говорящий 2: Да, согласна, что нужно изучить.',
        'Говорящий 1: Ну, я пока могу только сказать, что вроде восьмая прям осторожничает. Вторая в начале тоже не очень-то. А вот пятая вторым шагом, она прям огромную ставку сделала. Видимо, возможно, они так же будут делать в следующий раз. Знать бы еще, с кем мы будем играть. Ну, это она сделала, когда у нее был один шаг до победы. Просто даже в этот момент пытается безопасно сделать ход. Кто-то сразу готов.',
        'Говорящий 1: Да, мы таким же составом.',

        'Говорящий 1: Что, мы снова ставим 1?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 2: Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',
        'Говорящий 1: Мне кажется, надо меньше 10 тоже, я согласна. Это хорошо работает, когда они жгут ресурсы, и потом мы с ними работаем. Мне кажется, или 2, или... Я, конечно, за 2, чтобы... они осторожные. Но вторая, вначале у них была 23 ставка, то есть они настолько осторожные.',
        'Говорящий 2: Я не могу найти ссылку, куда ставить ставку.',
        'Говорящий 1: Сейчас скину.',

        'Говорящий 3: Предлагаю поставить полностью ставку. Сколько осталось. И мы их выиграем. Они не поставят полностью всю.',
        'Говорящий 1: О, она здорово у вас голос. Тебя слышно. Я поддерживаю, на самом деле.',
        'Говорящий 2: Да, абсолютно я согласна, что надо поставить максимум.',
        'Говорящий 3: Их надо скинуть 100%.',
        'Говорящий 1: Галина, ставка идеальна, как вы угадали.',
        'Говорящий 2: Вообще, я не знаю. Это какое-то подсознание. Я не знаю, даже как это.',
        'Говорящий 1: Класс.',
        'Говорящий 2: Так, 47. Все, 47 ставка, все отлично. 47 же, я же не ошибаюсь.',
        'Говорящий 1: Да.',
        'Говорящий 2: Так, отлично, все. Все, ставки сделаны.',

        'Говорящий 1: В финале будет интересно.',
        'Говорящий 2: Да, разворачиваются события, конечно.',
        'Говорящий 1: Но я всех поздравляю.',
        'Говорящий 2: Да, ну и точно. Я в какой-то момент подумала о том, что если они сейчас поставят 48 и все. Думаю, интересно, рискнут они или нет.',
        'Говорящий 1: Да, но тут как бы не рискнешь и не победишь, наверное. Четвертая рисковая, окей. Ну у меня зафиксировано, что они начали с 4. Правда, четвертая, по-моему.',
        'Говорящий 2: Да, и они вообще всю игру делают. Там 46, 42, у них все четные цифры. Я обратила просто внимание.',
        'Говорящий 1: Это хорошее наблюдение.',
        'Говорящий 1: Так, пятая 1, потом 11. Запишем.',

        'Говорящий 1: Три? Ну, что-нибудь такое, да, наверное.',
        'Говорящий 2: Я заметила, что если начинать примерно с мелких ставок, либо с 13, и потом ставить поменьше. Ну, короче, если начинать с 5 или 3, а потом десятые прибавлять, либо наоборот, сначала с десятых, там больше 10 ставить, например 11 или 13, а потом уменьшать единицы. Ну, то есть, тут надо тактику.',

        'Говорящий 1: Ну что, три, как договаривались? Пятые вроде начали в прошлый раз с 11.',
        'Говорящий 3: Да, давайте три, чтобы больше ресурсов осталось.',
        'Говорящий 2: Да-да-да, будем в стройке.',
        'Говорящий 1: Хорошо. 30 секунд это, конечно, жестко будет.',
        'Говорящий 2: Да, очень. Это вообще очень мало времени. Я успеваю только. Ну, жаль. У нас время почему-то не идет.',
        'Говорящий 1: Видимо, еще не все присоединились.',
        'Говорящий 2: Ну все, я уже тройку отправила. Ладно, согласна с вами. Надо выиграть.',
        'Говорящий 1: Чувствую ощущение, что в этом раунде будут больше рисковать.',
        'Говорящий 2: Угу.',
        'Говорящий 1: Ну, посмотрим. Будет смешно, если не поставить четыре.',
        'Говорящий 2: Это да.',

        'Говорящий 1: Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
        'Говорящий 3: Да, мне тоже кажется, что они сейчас пойдут в ва-банк, они полностью поставят, и они тогда выиграют.',
        'Говорящий 1: Тогда надо 45.',
        'Говорящий 3: У нас сколько осталось?',
        'Говорящий 2: У нас осталось 47.',
        'Говорящий 3: 47.',
        'Говорящий 2: Это же следовательно, что они будут рисковать 100%.',
        'Говорящий 3: Да.',
        'Говорящий 1: Давайте 45 тогда, да?',
        'Говорящий 3: Я думаю, да.',
        'Говорящий 1: И у нас останется на два хода перечень, если что.',
        'Говорящий 2: Ну да.',
        'Говорящий 1: Надеюсь, они не подумают, что мы решим так сделать. А то эти эффективные игры.',
        'Говорящий 3: 45.',
        'Говорящий 1: 45 вроде, да.',
        'Говорящий 2: У меня 45 и только ссылочка, пришлите еще раз кого-то.',
        'Говорящий 1: Сейчас.',
        'Говорящий 2: Я не успела ставку сделать, верните, пожалуйста.',

        'Говорящий 1: Из нас уже никто не выиграет.',
        'Говорящий 2: Я предлагаю ставить единицу и как бы оставить... Ну, то есть ставить единицу уже тогда.',
        'Говорящий 1: Ну, придётся, видимо. Потому что если двойку, то... Ну, всё равно... Ну, короче, уже на два хода ни у кого не останется.',
        'Говорящий 1: Давайте единицу.',

        'Говорящий 3: Ну, выбора у нас особого нет.',
        'Говорящий 1: Ну да. Интересный финал. Единица, так единица.',
        'Говорящий 3: Что у нас, камень остается внизу, то победитель или нет? Или я что-то путаю?',
        'Говорящий 1: Ну, он остается на второй ступени, а победителя нет, да.',
        'Говорящий 3: То есть камень упадет вниз, вниз со второй ступени, потому что одинаковая будет ставка.',
        'Говорящий 1: Ну, он останется на второй, но это не засчитывается как победа пятой команды, потому что до вершины его не добили.',
        'Говорящий 3: Ну, понятно. Спасибо. Мы молодцы, мы не даем выиграть.',
        'Говорящий 1: Ну, блин, это... Этот ход, когда 45, конечно. Интересный был.',
]

### Обработка поступаемой реплики

In [ ]:
#test = [test]

for i in test:
    speaker, utterance = i.split(': ')
    speaker = speaker.strip() # 'Speaker A'
    utterance = utterance.strip() # '*text*'

    # приводим реплику в нормальный вид и веторизуем

    utterance_preprocessed = preprocess(utterance, stop_words, punctuation_marks, morph)
    utterance_seq = text_to_sequence(utterance_preprocessed, word_to_index)
    utterance_bow = vectorize_sequences([utterance_seq], max_words)

    # предсказываем категорию высказывания

    prediction = lr.predict(utterance_bow)

    # добавляем спикера в словарь (если его еще нет)

    if speaker not in speakers:
      speakers[speaker] = []

    # если классификатор распознал реплику как предложение, присваиваем ее спикеру

    if prediction:
      speakers[speaker].append(utterance)

In [ ]:
speakers

{'Говорящий 1': ['А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
  'Что мы тогда ставим, 21 или другие варианты есть?',
  'Нет, друзья, у них максимум 20 ставка, нам же не нужно 29. У нас 21 это максимум.',
  'Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
  'У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
  'Ну, здесь я просто не знаю, насколько велика вероятность, что они поставят меньше 7.',
  'Что, мы снова ставим 1?',
  'Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
  'Давайте 45 тогда, да?',
  'Давайте единицу.'],
 'Говорящий 2': ['Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет 

In [ ]:
for key, values in speakers.items():
  print(len(values))

9
13
1


# С помощью векторных представлений слов на обученных моделях

Используем логистическую регрессию и векторные представления Navec.

### Установка и загрузка библиотек

In [ ]:
!pip install pymorphy3
!pip install navec

In [ ]:
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from navec import Navec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Константы

In [ ]:
random_state = 42
max_review_len = 50
vector_size = 300

## Загружаем и готовим набор данных

In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Classes/file_upd.csv')
dataframe.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
dataframe

,text,category
0,Давайте двойку.,предложение
1,Ставлю.,предложение
2,Нам надо пять поставить.,предложение
3,"Ребят, 2 надо ставить.",предложение
4,"2 надо ставить, у нас ресурсов нет.",предложение
...,...,...
260,То есть у нас сейчас получается будет два раун...,другое
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое
263,Давай,другое


In [ ]:
dataframe['preprocessed_text'] = dataframe.apply(lambda row: preprocess(row['text'], stop_words, punctuation_marks, morph), axis=1)

In [ ]:
dataframe

,text,category,preprocessed_text
0,Давайте двойку.,предложение,"[давать, двойка]"
1,Ставлю.,предложение,[ставить]
2,Нам надо пять поставить.,предложение,"[пять, поставить]"
3,"Ребят, 2 надо ставить.",предложение,"[ребята, 2, ставить]"
4,"2 надо ставить, у нас ресурсов нет.",предложение,"[2, ставить, ресурс]"
...,...,...,...
260,То есть у нас сейчас получается будет два раун...,другое,"[получаться, раунд]"
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое,"[смотреть, наблюдать, стратегия]"
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое,"[варвар, каждый, отправить, ещё, ответ, следую..."
263,Давай,другое,[давать]


## Загружаем предварительно обученные векторы Navec

In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

--2024-05-30 09:32:02--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  19.7MB/s    in 2.6s    

2024-05-30 09:32:05 (19.7 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

### Векторизуем текст

Функция для векторизации текста

In [ ]:
def vectorize_text(txt, navec, max_review_len):
    unk = navec['<unk>']
    text_embeddings = []
    for tocken in txt:
        embedding = navec.get(tocken, unk)
        text_embeddings.append(embedding)
    # Дополняем или обрезаем реплики для фиксированной длины max_review_len
    l = len(text_embeddings)
    if l > max_review_len:
        text_embeddings = text_embeddings[:max_review_len]
    else:
        text_embeddings.extend([navec['<pad>']] * (max_review_len - l))
    return text_embeddings

Применяем функцию векторизации текста к набору данных

In [ ]:
dataframe['embeddings'] = dataframe.apply(lambda row: vectorize_text(row['preprocessed_text'], navec, max_review_len), axis=1)

In [ ]:
dataframe

,text,category,preprocessed_text,embeddings
0,Давайте двойку.,предложение,"[давать, двойка]","[[-0.20394225, -0.5428032, 0.17268088, -0.5472..."
1,Ставлю.,предложение,[ставить],"[[-0.56923324, -0.3954833, -0.06762549, 0.0721..."
2,Нам надо пять поставить.,предложение,"[пять, поставить]","[[0.1683296, -0.4029474, -0.67731494, 0.000809..."
3,"Ребят, 2 надо ставить.",предложение,"[ребята, 2, ставить]","[[0.56820583, 0.046246283, -0.28193244, 0.5412..."
4,"2 надо ставить, у нас ресурсов нет.",предложение,"[2, ставить, ресурс]","[[0.21431214, 0.37028718, 0.13679631, -0.18653..."
...,...,...,...,...
260,То есть у нас сейчас получается будет два раун...,другое,"[получаться, раунд]","[[0.010760385, 0.26762047, 0.40114763, 0.14934..."
261,"Смотрим их, да, и наблюдаем за стратегией других.",другое,"[смотреть, наблюдать, стратегия]","[[0.36390617, -0.5678413, -0.24011786, 0.23606..."
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",другое,"[варвар, каждый, отправить, ещё, ответ, следую...","[[-0.55413014, -0.12571354, 0.102926634, 0.271..."
263,Давай,другое,[давать],"[[-0.20394225, -0.5428032, 0.17268088, -0.5472..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [ ]:
labels = {'предложение': 1, 'другое': 0}

In [ ]:
dataframe.replace({'category': labels}, inplace=True)

In [ ]:
dataframe

,text,category,preprocessed_text,embeddings
0,Давайте двойку.,1,"[давать, двойка]","[[-0.20394225, -0.5428032, 0.17268088, -0.5472..."
1,Ставлю.,1,[ставить],"[[-0.56923324, -0.3954833, -0.06762549, 0.0721..."
2,Нам надо пять поставить.,1,"[пять, поставить]","[[0.1683296, -0.4029474, -0.67731494, 0.000809..."
3,"Ребят, 2 надо ставить.",1,"[ребята, 2, ставить]","[[0.56820583, 0.046246283, -0.28193244, 0.5412..."
4,"2 надо ставить, у нас ресурсов нет.",1,"[2, ставить, ресурс]","[[0.21431214, 0.37028718, 0.13679631, -0.18653..."
...,...,...,...,...
260,То есть у нас сейчас получается будет два раун...,0,"[получаться, раунд]","[[0.010760385, 0.26762047, 0.40114763, 0.14934..."
261,"Смотрим их, да, и наблюдаем за стратегией других.",0,"[смотреть, наблюдать, стратегия]","[[0.36390617, -0.5678413, -0.24011786, 0.23606..."
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",0,"[варвар, каждый, отправить, ещё, ответ, следую...","[[-0.55413014, -0.12571354, 0.102926634, 0.271..."
263,Давай,0,[давать],"[[-0.20394225, -0.5428032, 0.17268088, -0.5472..."


### Разделяем метки классов и данные для обучения

Данные для обучения

In [ ]:
x_vectors = np.array(dataframe['embeddings'].tolist()).reshape(len(dataframe), vector_size * max_review_len)
y_train = dataframe['category']

In [ ]:
x_vectors.shape

(265, 15000)

Данные для тестирования

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

# Инициализация модели логистической регрессии
lr = LogisticRegression(random_state = random_state)

# Создание скорера для f1-score
f1_scorer = make_scorer(f1_score)

# Выполнение кросс-валидации с вычислением f1-score
f1_scores = cross_val_score(lr, x_vectors, y_train, cv=5, scoring=f1_scorer)

# Вывод результатов
print("Средний F1-счет: %0.2f (+/- %0.2f)" % (f1_scores.mean(), f1_scores.std() * 2))


Средний F1-счет: 0.63 (+/- 0.19)


## Создаем и обучаем модель машинного обучения

In [ ]:
lr.fit(x_vectors, y_train)

LogisticRegression(random_state=42)

## Применяем модель для определения категории высказывания

In [ ]:
speakers = dict()

In [ ]:
text = ['Говорящий 1: Всем привет. Мы, кажется, все собрались, да? У нас должно было быть четверо. Меня хорошо слышно? Хорошо. Я, простите, больше не буду без камеры. У нас сейчас важный такт. Нам нужно выбрать, кто из нас будет подавать заявки в форме. И я пришлю форму.',
        'Говорящий 2: Я буду делать.',
        'Говорящий 1: Отлично, давайте. Сейчас вам в чате зума. Все, получили? А сколько у нас минут заполнено?',
        'Говорящий 2: Пять минут.',
        'Говорящий 1: Ага. Тогда какие предложения по ставкам?',
        'Говорящий 2: Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
        'Говорящий 1: Ну да, мне тоже кажется, что или надо больше половины, чтобы сразу на свою сторону, или минимальную, чтобы другая команда ресурсы пожгла.',
        'Говорящий 2: Угу. Так. Проверочный код вашей команды. А где его проверочный код получить?',
        'Говорящий 1: А вот тоже я отправила сообщение тут же.',
        'Говорящий 1: А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
        'Говорящий 2: Ну 13, да, интересно в принципе ставка, если с учетом того, что она заканчивается на неудобную цифру. Так, а у нас играет кто получается?',
        'Говорящий 1: Мы наверное с четвертой играем.',
        'Говорящий 2: У нас осталось 1 минута. Так, давайте, кто за единицу голосует.',
        'Говорящий 1: Я за единицу. Ага.',
        'Говорящий 2: Я за единицу. И... Я за единицу.',
        'Говорящий 2: Так, отлично. Так, единица или 13?',
        'Говорящий 1: Галина, я вас плохо слышу, если честно.',
        'Говорящий 2: Так, а мы разговариваем, Милана, с вами, да, вдвоем?',
        'Говорящий 1: Я так понимаю, у Анны что-то с микрофоном? Юлия, вы с нами?',
        'Говорящий 2: Давайте, все, отправляю единицу. Ставку сделали. Нужно выйти, да, теперь?',
        'Говорящий 1: Она сама отправится.',
        'Говорящий 2: А у меня пришло приглашение выйти из зала.',
        'Говорящий 1: О, отлично. Спасибо, Юлия. Галина, вы с нами? Все, мы все теперь в чате. Хорошо.',
        'Говорящий 1: Что мы тогда ставим, 21 или другие варианты есть?',
        'Говорящий 1: Так, Юлия Анна пишет 21, Галина 29.',
        'Говорящий 1: Нет, друзья, у них максимум 20 ставка, нам же не нужно 29. У нас 21 это максимум.',
        'Говорящий 1: Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
        'Говорящий 1: Да, точно. У всех, видимо, что-то со звуком. Ничего, я буду озвучивать.',
        'Говорящий 2: Так, все здесь?',
        'Говорящий 1: Здесь.',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 1: У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
        'Говорящий 2: Ну, смотрите, а это же будет последний ход?',
        'Говорящий 1: Что?',
        'Говорящий 1: Это же будет последний раунд?',
        'Говорящий 1: Нет, мы играем пока не перетащим камень или пока не закончатся монеты.',
        'Говорящий 2: Но я предлагаю 7. Почему? Потому что даже если они поставят чуть больше 8, у них в остатке останется 11, а у нас тут чуть больше монет 21.',
        'Говорящий 1: Ну, здесь я просто не знаю, насколько велика вероятность, что они поставят меньше 7.',
        'Говорящий 2: Нет, ну тут меньше 7 они точно не поставят. Хотя, с другой стороны.',
        'Говорящий 1: Мы можем поставить чуть больше половины. Просто тогда будет вероятность больше, что мы...',
        'Говорящий 2: Если они перетянут, допустим, если они поставят 8, и  них в остатке останется 11, а у нас 21, то у нас будет преимущество на следующие шаги, у нас останется чуть больше монет. Просто если они сейчас поставят 8, мы поставим 10, и мы перетянем камень, у нас останется 18.',
        'Говорящий 1: Поздравляю, в одном шаге от победы.',
        'Говорящий 2: Так, я предлагаю поставить 7.',
        'Говорящий 1: Так мы можем все поставить, мы уже точно их выиграем, если поставим 18. У них меньше. Так что ставим 18 и все. Камень наш.',
        'Говорящий 2: А, ну да, логично. Так, а почему-то у меня не возвращается моя ставка.',
        'Говорящий 1: 18 у нас же, правильно?',
        'Говорящий 2: Да, я ставлю 18, конечно, заканчиваем.',
        'Говорящий 1: Супер.',
        'Говорящий 2: 18, правильно.',
        'Говорящий 1: Интересно, если бы мы поставили 15, мы бы вышли с монетами?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Ну, очень, как будто слабо. Сейчас слышно.',
        'Говорящий 2: А что нам делать-то нужно?',
        'Говорящий 1: Пока мы отдыхаем. И ждем следующий раунд.',
        'Говорящий 2: Я поняла.',
        'Говорящий 1: Можем, не знаю, подумать, что... Может быть, в следующий раунд, короче, на следующем ходе, можно зафиксировать, какие стратегии команды-победители используют, чтобы примерно знать, может быть, с кем может столкнуться в следующем раунде. Посмотреть пока будет демонстрировать.',
        'Говорящий 2: Да, согласна, что нужно изучить.',
        'Говорящий 1: Ну, я пока могу только сказать, что вроде восьмая прям осторожничает. Вторая в начале тоже не очень-то. А вот пятая вторым шагом, она прям огромную ставку сделала. Видимо, возможно, они так же будут делать в следующий раз. Знать бы еще, с кем мы будем играть. Ну, это она сделала, когда у нее был один шаг до победы. Просто даже в этот момент пытается безопасно сделать ход. Кто-то сразу готов.',
        'Говорящий 1: Да, мы таким же составом.',

        'Говорящий 1: Что, мы снова ставим 1?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 2: Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',
        'Говорящий 1: Мне кажется, надо меньше 10 тоже, я согласна. Это хорошо работает, когда они жгут ресурсы, и потом мы с ними работаем. Мне кажется, или 2, или... Я, конечно, за 2, чтобы... они осторожные. Но вторая, вначале у них была 23 ставка, то есть они настолько осторожные.',
        'Говорящий 2: Я не могу найти ссылку, куда ставить ставку.',
        'Говорящий 1: Сейчас скину.',

        'Говорящий 3: Предлагаю поставить полностью ставку. Сколько осталось. И мы их выиграем. Они не поставят полностью всю.',
        'Говорящий 1: О, она здорово у вас голос. Тебя слышно. Я поддерживаю, на самом деле.',
        'Говорящий 2: Да, абсолютно я согласна, что надо поставить максимум.',
        'Говорящий 3: Их надо скинуть 100%.',
        'Говорящий 1: Галина, ставка идеальна, как вы угадали.',
        'Говорящий 2: Вообще, я не знаю. Это какое-то подсознание. Я не знаю, даже как это.',
        'Говорящий 1: Класс.',
        'Говорящий 2: Так, 47. Все, 47 ставка, все отлично. 47 же, я же не ошибаюсь.',
        'Говорящий 1: Да.',
        'Говорящий 2: Так, отлично, все. Все, ставки сделаны.',

        'Говорящий 1: В финале будет интересно.',
        'Говорящий 2: Да, разворачиваются события, конечно.',
        'Говорящий 1: Но я всех поздравляю.',
        'Говорящий 2: Да, ну и точно. Я в какой-то момент подумала о том, что если они сейчас поставят 48 и все. Думаю, интересно, рискнут они или нет.',
        'Говорящий 1: Да, но тут как бы не рискнешь и не победишь, наверное. Четвертая рисковая, окей. Ну у меня зафиксировано, что они начали с 4. Правда, четвертая, по-моему.',
        'Говорящий 2: Да, и они вообще всю игру делают. Там 46, 42, у них все четные цифры. Я обратила просто внимание.',
        'Говорящий 1: Это хорошее наблюдение.',
        'Говорящий 1: Так, пятая 1, потом 11. Запишем.',

        'Говорящий 1: Три? Ну, что-нибудь такое, да, наверное.',
        'Говорящий 2: Я заметила, что если начинать примерно с мелких ставок, либо с 13, и потом ставить поменьше. Ну, короче, если начинать с 5 или 3, а потом десятые прибавлять, либо наоборот, сначала с десятых, там больше 10 ставить, например 11 или 13, а потом уменьшать единицы. Ну, то есть, тут надо тактику.',

        'Говорящий 1: Ну что, три, как договаривались? Пятые вроде начали в прошлый раз с 11.',
        'Говорящий 3: Да, давайте три, чтобы больше ресурсов осталось.',
        'Говорящий 2: Да-да-да, будем в стройке.',
        'Говорящий 1: Хорошо. 30 секунд это, конечно, жестко будет.',
        'Говорящий 2: Да, очень. Это вообще очень мало времени. Я успеваю только. Ну, жаль. У нас время почему-то не идет.',
        'Говорящий 1: Видимо, еще не все присоединились.',
        'Говорящий 2: Ну все, я уже тройку отправила. Ладно, согласна с вами. Надо выиграть.',
        'Говорящий 1: Чувствую ощущение, что в этом раунде будут больше рисковать.',
        'Говорящий 2: Угу.',
        'Говорящий 1: Ну, посмотрим. Будет смешно, если не поставить четыре.',
        'Говорящий 2: Это да.',

        'Говорящий 1: Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
        'Говорящий 3: Да, мне тоже кажется, что они сейчас пойдут в ва-банк, они полностью поставят, и они тогда выиграют.',
        'Говорящий 1: Тогда надо 45.',
        'Говорящий 3: У нас сколько осталось?',
        'Говорящий 2: У нас осталось 47.',
        'Говорящий 3: 47.',
        'Говорящий 2: Это же следовательно, что они будут рисковать 100%.',
        'Говорящий 3: Да.',
        'Говорящий 1: Давайте 45 тогда, да?',
        'Говорящий 3: Я думаю, да.',
        'Говорящий 1: И у нас останется на два хода перечень, если что.',
        'Говорящий 2: Ну да.',
        'Говорящий 1: Надеюсь, они не подумают, что мы решим так сделать. А то эти эффективные игры.',
        'Говорящий 3: 45.',
        'Говорящий 1: 45 вроде, да.',
        'Говорящий 2: У меня 45 и только ссылочка, пришлите еще раз кого-то.',
        'Говорящий 1: Сейчас.',
        'Говорящий 2: Я не успела ставку сделать, верните, пожалуйста.',

        'Говорящий 1: Из нас уже никто не выиграет.',
        'Говорящий 2: Я предлагаю ставить единицу и как бы оставить... Ну, то есть ставить единицу уже тогда.',
        'Говорящий 1: Ну, придётся, видимо. Потому что если двойку, то... Ну, всё равно... Ну, короче, уже на два хода ни у кого не останется.',
        'Говорящий 1: Давайте единицу.',

        'Говорящий 3: Ну, выбора у нас особого нет.',
        'Говорящий 1: Ну да. Интересный финал. Единица, так единица.',
        'Говорящий 3: Что у нас, камень остается внизу, то победитель или нет? Или я что-то путаю?',
        'Говорящий 1: Ну, он остается на второй ступени, а победителя нет, да.',
        'Говорящий 3: То есть камень упадет вниз, вниз со второй ступени, потому что одинаковая будет ставка.',
        'Говорящий 1: Ну, он останется на второй, но это не засчитывается как победа пятой команды, потому что до вершины его не добили.',
        'Говорящий 3: Ну, понятно. Спасибо. Мы молодцы, мы не даем выиграть.',
        'Говорящий 1: Ну, блин, это... Этот ход, когда 45, конечно. Интересный был.',
]

In [ ]:
#test = [test]

for i in text:
    speaker, utterance = str(i).split(': ')
    speaker = speaker.strip() # 'Speaker A'
    utterance = utterance.strip() # '*text*'

    # приводим реплику в нормальный вид и веторизуем

    utterance_preprocessed = preprocess(utterance, stop_words, punctuation_marks, morph)
    utterance_vectorized_text = vectorize_text(utterance_preprocessed, navec, max_review_len)
    utterance_vector = np.array(utterance_vectorized_text).reshape(1, vector_size * max_review_len)

    # предсказываем категорию высказывания

    prediction = lr.predict(utterance_vector)

    # добавляем спикера в словарь (если его еще нет)

    if speaker not in speakers:
      speakers[speaker] = []

    # если классификатор распознал реплику как предложение, присваиваем ее спикеру

    if prediction:
      speakers[speaker].append(utterance)


In [ ]:
speakers

{'Говорящий 1': ['А вот тоже я отправила сообщение тут же.',
  'А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
  'Что мы тогда ставим, 21 или другие варианты есть?',
  'У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
  'Мы можем поставить чуть больше половины. Просто тогда будет вероятность больше, что мы...',
  'Да, но тут как бы не рискнешь и не победишь, наверное. Четвертая рисковая, окей. Ну у меня зафиксировано, что они начали с 4. Правда, четвертая, по-моему.',
  'Ну что, три, как договаривались? Пятые вроде начали в прошлый раз с 11.',
  'Давайте 45 тогда, да?',
  'Давайте единицу.'],
 'Говорящий 2': ['У нас осталось 1 минута. Так, давайте, кто за единицу голосует.',
  'Давайте, все, отправляю единицу. Ставку сделали. Нужно выйти, да, теперь?',
  'Но я предлагаю 7. Почему? Потому что даже если они поставят чуть больше 8, у них в остатке остан

# Обучение Word2Vec на своих данных

### Загружаем данные и предобрабатываем их

In [ ]:
mapping = {'предложение': 1, 'другое': 0}

In [ ]:
daf = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Classes/file_upd.csv")
daf.drop('Unnamed: 0', inplace=True, axis=1)
daf['preprocessed_text'] = daf.apply(lambda row: preprocess(row['text'], stop_words, punctuation_marks, morph), axis=1)
daf.replace({'category': mapping}, inplace=True)

daf

,text,category,preprocessed_text
0,Давайте двойку.,1,"[давать, двойка]"
1,Ставлю.,1,[ставить]
2,Нам надо пять поставить.,1,"[пять, поставить]"
3,"Ребят, 2 надо ставить.",1,"[ребята, 2, ставить]"
4,"2 надо ставить, у нас ресурсов нет.",1,"[2, ставить, ресурс]"
...,...,...,...
260,То есть у нас сейчас получается будет два раун...,0,"[получаться, раунд]"
261,"Смотрим их, да, и наблюдаем за стратегией других.",0,"[смотреть, наблюдать, стратегия]"
262,"Так, а мне, Варвара, мне надо каждый раз отпра...",0,"[варвар, каждый, отправить, ещё, ответ, следую..."
263,Давай,0,[давать]


In [ ]:
import gensim.models

In [ ]:
model = gensim.models.Word2Vec(sentences=daf['preprocessed_text'],
                               min_count=5,
                               vector_size=50)

In [ ]:
model.wv['смотреть']

array([ 0.00525098, -0.01646165,  0.0156452 ,  0.01700614, -0.01882887,
        0.00457133,  0.02023043, -0.01463925, -0.01414941, -0.01551547,
        0.01678299, -0.00223147,  0.01838414, -0.01586233,  0.0076463 ,
        0.00569293,  0.00147755,  0.0049887 , -0.01557483, -0.01923255,
        0.00489978,  0.01281673,  0.01628518,  0.01120741, -0.00127193,
        0.01676711, -0.01896304,  0.0067689 ,  0.00020135,  0.00766857,
        0.01500603, -0.01355894,  0.01118715, -0.01914207, -0.00187387,
       -0.01714657, -0.00964785,  0.01879859, -0.00335039,  0.00556938,
        0.01851103,  0.01748915, -0.01630106, -0.00623814,  0.02059056,
        0.01047766, -0.0033963 , -0.01781583,  0.00621599, -0.01327886],
      dtype=float32)

In [ ]:
model.wv.most_similar('давать')

[('скажем', 0.3347277343273163),
 ('короткий', 0.32504501938819885),
 ('вид', 0.3073335587978363),
 ('кто-то', 0.27859339118003845),
 ('играть', 0.2782719135284424),
 ('начать', 0.2782103419303894),
 ('40', 0.2645968496799469),
 ('внизу', 0.2641281187534332),
 ('хороший', 0.25802162289619446),
 ('стратегия', 0.2532322406768799)]

In [ ]:
model.save('word2vec-classification-50')

### Загрузка модели и ее применение

In [ ]:
speakers = dict()

text = ['Говорящий 1: Всем привет. Мы, кажется, все собрались, да? У нас должно было быть четверо. Меня хорошо слышно? Хорошо. Я, простите, больше не буду без камеры. У нас сейчас важный такт. Нам нужно выбрать, кто из нас будет подавать заявки в форме. И я пришлю форму.',
        'Говорящий 2: Я буду делать.',
        'Говорящий 1: Отлично, давайте. Сейчас вам в чате зума. Все, получили? А сколько у нас минут заполнено?',
        'Говорящий 2: Пять минут.',
        'Говорящий 1: Ага. Тогда какие предложения по ставкам?',
        'Говорящий 2: Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
        'Говорящий 1: Ну да, мне тоже кажется, что или надо больше половины, чтобы сразу на свою сторону, или минимальную, чтобы другая команда ресурсы пожгла.',
        'Говорящий 2: Угу. Так. Проверочный код вашей команды. А где его проверочный код получить?',
        'Говорящий 1: А вот тоже я отправила сообщение тут же.',
        'Говорящий 1: А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
        'Говорящий 2: Ну 13, да, интересно в принципе ставка, если с учетом того, что она заканчивается на неудобную цифру. Так, а у нас играет кто получается?',
        'Говорящий 1: Мы наверное с четвертой играем.',
        'Говорящий 2: У нас осталось 1 минута. Так, давайте, кто за единицу голосует.',
        'Говорящий 1: Я за единицу. Ага.',
        'Говорящий 2: Я за единицу. И... Я за единицу.',
        'Говорящий 2: Так, отлично. Так, единица или 13?',
        'Говорящий 1: Галина, я вас плохо слышу, если честно.',
        'Говорящий 2: Так, а мы разговариваем, Милана, с вами, да, вдвоем?',
        'Говорящий 1: Я так понимаю, у Анны что-то с микрофоном? Юлия, вы с нами?',
        'Говорящий 2: Давайте, все, отправляю единицу. Ставку сделали. Нужно выйти, да, теперь?',
        'Говорящий 1: Она сама отправится.',
        'Говорящий 2: А у меня пришло приглашение выйти из зала.',
        'Говорящий 1: О, отлично. Спасибо, Юлия. Галина, вы с нами? Все, мы все теперь в чате. Хорошо.',
        'Говорящий 1: Что мы тогда ставим, 21 или другие варианты есть?',
        'Говорящий 1: Так, Юлия Анна пишет 21, Галина 29.',
        'Говорящий 1: Нет, друзья, у них максимум 20 ставка, нам же не нужно 29. У нас 21 это максимум.',
        'Говорящий 1: Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
        'Говорящий 1: Да, точно. У всех, видимо, что-то со звуком. Ничего, я буду озвучивать.',
        'Говорящий 2: Так, все здесь?',
        'Говорящий 1: Здесь.',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 1: У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
        'Говорящий 2: Ну, смотрите, а это же будет последний ход?',
        'Говорящий 1: Что?',
        'Говорящий 1: Это же будет последний раунд?',
        'Говорящий 1: Нет, мы играем пока не перетащим камень или пока не закончатся монеты.',
        'Говорящий 2: Но я предлагаю 7. Почему? Потому что даже если они поставят чуть больше 8, у них в остатке останется 11, а у нас тут чуть больше монет 21.',
        'Говорящий 1: Ну, здесь я просто не знаю, насколько велика вероятность, что они поставят меньше 7.',
        'Говорящий 2: Нет, ну тут меньше 7 они точно не поставят. Хотя, с другой стороны.',
        'Говорящий 1: Мы можем поставить чуть больше половины. Просто тогда будет вероятность больше, что мы...',
        'Говорящий 2: Если они перетянут, допустим, если они поставят 8, и  них в остатке останется 11, а у нас 21, то у нас будет преимущество на следующие шаги, у нас останется чуть больше монет. Просто если они сейчас поставят 8, мы поставим 10, и мы перетянем камень, у нас останется 18.',
        'Говорящий 1: Поздравляю, в одном шаге от победы.',
        'Говорящий 2: Так, я предлагаю поставить 7.',
        'Говорящий 1: Так мы можем все поставить, мы уже точно их выиграем, если поставим 18. У них меньше. Так что ставим 18 и все. Камень наш.',
        'Говорящий 2: А, ну да, логично. Так, а почему-то у меня не возвращается моя ставка.',
        'Говорящий 1: 18 у нас же, правильно?',
        'Говорящий 2: Да, я ставлю 18, конечно, заканчиваем.',
        'Говорящий 1: Супер.',
        'Говорящий 2: 18, правильно.',
        'Говорящий 1: Интересно, если бы мы поставили 15, мы бы вышли с монетами?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Ну, очень, как будто слабо. Сейчас слышно.',
        'Говорящий 2: А что нам делать-то нужно?',
        'Говорящий 1: Пока мы отдыхаем. И ждем следующий раунд.',
        'Говорящий 2: Я поняла.',
        'Говорящий 1: Можем, не знаю, подумать, что... Может быть, в следующий раунд, короче, на следующем ходе, можно зафиксировать, какие стратегии команды-победители используют, чтобы примерно знать, может быть, с кем может столкнуться в следующем раунде. Посмотреть пока будет демонстрировать.',
        'Говорящий 2: Да, согласна, что нужно изучить.',
        'Говорящий 1: Ну, я пока могу только сказать, что вроде восьмая прям осторожничает. Вторая в начале тоже не очень-то. А вот пятая вторым шагом, она прям огромную ставку сделала. Видимо, возможно, они так же будут делать в следующий раз. Знать бы еще, с кем мы будем играть. Ну, это она сделала, когда у нее был один шаг до победы. Просто даже в этот момент пытается безопасно сделать ход. Кто-то сразу готов.',
        'Говорящий 1: Да, мы таким же составом.',

        'Говорящий 1: Что, мы снова ставим 1?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 2: Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',
        'Говорящий 1: Мне кажется, надо меньше 10 тоже, я согласна. Это хорошо работает, когда они жгут ресурсы, и потом мы с ними работаем. Мне кажется, или 2, или... Я, конечно, за 2, чтобы... они осторожные. Но вторая, вначале у них была 23 ставка, то есть они настолько осторожные.',
        'Говорящий 2: Я не могу найти ссылку, куда ставить ставку.',
        'Говорящий 1: Сейчас скину.',

        'Говорящий 3: Предлагаю поставить полностью ставку. Сколько осталось. И мы их выиграем. Они не поставят полностью всю.',
        'Говорящий 1: О, она здорово у вас голос. Тебя слышно. Я поддерживаю, на самом деле.',
        'Говорящий 2: Да, абсолютно я согласна, что надо поставить максимум.',
        'Говорящий 3: Их надо скинуть 100%.',
        'Говорящий 1: Галина, ставка идеальна, как вы угадали.',
        'Говорящий 2: Вообще, я не знаю. Это какое-то подсознание. Я не знаю, даже как это.',
        'Говорящий 1: Класс.',
        'Говорящий 2: Так, 47. Все, 47 ставка, все отлично. 47 же, я же не ошибаюсь.',
        'Говорящий 1: Да.',
        'Говорящий 2: Так, отлично, все. Все, ставки сделаны.',

        'Говорящий 1: В финале будет интересно.',
        'Говорящий 2: Да, разворачиваются события, конечно.',
        'Говорящий 1: Но я всех поздравляю.',
        'Говорящий 2: Да, ну и точно. Я в какой-то момент подумала о том, что если они сейчас поставят 48 и все. Думаю, интересно, рискнут они или нет.',
        'Говорящий 1: Да, но тут как бы не рискнешь и не победишь, наверное. Четвертая рисковая, окей. Ну у меня зафиксировано, что они начали с 4. Правда, четвертая, по-моему.',
        'Говорящий 2: Да, и они вообще всю игру делают. Там 46, 42, у них все четные цифры. Я обратила просто внимание.',
        'Говорящий 1: Это хорошее наблюдение.',
        'Говорящий 1: Так, пятая 1, потом 11. Запишем.',

        'Говорящий 1: Три? Ну, что-нибудь такое, да, наверное.',
        'Говорящий 2: Я заметила, что если начинать примерно с мелких ставок, либо с 13, и потом ставить поменьше. Ну, короче, если начинать с 5 или 3, а потом десятые прибавлять, либо наоборот, сначала с десятых, там больше 10 ставить, например 11 или 13, а потом уменьшать единицы. Ну, то есть, тут надо тактику.',

        'Говорящий 1: Ну что, три, как договаривались? Пятые вроде начали в прошлый раз с 11.',
        'Говорящий 3: Да, давайте три, чтобы больше ресурсов осталось.',
        'Говорящий 2: Да-да-да, будем в стройке.',
        'Говорящий 1: Хорошо. 30 секунд это, конечно, жестко будет.',
        'Говорящий 2: Да, очень. Это вообще очень мало времени. Я успеваю только. Ну, жаль. У нас время почему-то не идет.',
        'Говорящий 1: Видимо, еще не все присоединились.',
        'Говорящий 2: Ну все, я уже тройку отправила. Ладно, согласна с вами. Надо выиграть.',
        'Говорящий 1: Чувствую ощущение, что в этом раунде будут больше рисковать.',
        'Говорящий 2: Угу.',
        'Говорящий 1: Ну, посмотрим. Будет смешно, если не поставить четыре.',
        'Говорящий 2: Это да.',

        'Говорящий 1: Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
        'Говорящий 3: Да, мне тоже кажется, что они сейчас пойдут в ва-банк, они полностью поставят, и они тогда выиграют.',
        'Говорящий 1: Тогда надо 45.',
        'Говорящий 3: У нас сколько осталось?',
        'Говорящий 2: У нас осталось 47.',
        'Говорящий 3: 47.',
        'Говорящий 2: Это же следовательно, что они будут рисковать 100%.',
        'Говорящий 3: Да.',
        'Говорящий 1: Давайте 45 тогда, да?',
        'Говорящий 3: Я думаю, да.',
        'Говорящий 1: И у нас останется на два хода перечень, если что.',
        'Говорящий 2: Ну да.',
        'Говорящий 1: Надеюсь, они не подумают, что мы решим так сделать. А то эти эффективные игры.',
        'Говорящий 3: 45.',
        'Говорящий 1: 45 вроде, да.',
        'Говорящий 2: У меня 45 и только ссылочка, пришлите еще раз кого-то.',
        'Говорящий 1: Сейчас.',
        'Говорящий 2: Я не успела ставку сделать, верните, пожалуйста.',

        'Говорящий 1: Из нас уже никто не выиграет.',
        'Говорящий 2: Я предлагаю ставить единицу и как бы оставить... Ну, то есть ставить единицу уже тогда.',
        'Говорящий 1: Ну, придётся, видимо. Потому что если двойку, то... Ну, всё равно... Ну, короче, уже на два хода ни у кого не останется.',
        'Говорящий 1: Давайте единицу.',

        'Говорящий 3: Ну, выбора у нас особого нет.',
        'Говорящий 1: Ну да. Интересный финал. Единица, так единица.',
        'Говорящий 3: Что у нас, камень остается внизу, то победитель или нет? Или я что-то путаю?',
        'Говорящий 1: Ну, он остается на второй ступени, а победителя нет, да.',
        'Говорящий 3: То есть камень упадет вниз, вниз со второй ступени, потому что одинаковая будет ставка.',
        'Говорящий 1: Ну, он останется на второй, но это не засчитывается как победа пятой команды, потому что до вершины его не добили.',
        'Говорящий 3: Ну, понятно. Спасибо. Мы молодцы, мы не даем выиграть.',
        'Говорящий 1: Ну, блин, это... Этот ход, когда 45, конечно. Интересный был.',
        'Говорящий 3: Давай, я согласна!'
]

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

# Загрузка предобученной модели Word2Vec
model = Word2Vec.load('/content/drive/MyDrive/Colab_Notebooks/models/word2vec_class/word2vec-classification-50')

# Функция для получения среднего вектора для документа
def document_vector(word2vec_model, doc):
    # Отфильтровываем слова, которых нет в модели
    doc = [word for word in doc if word in word2vec_model.wv]
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size, dtype=np.float32)

    # Считаем среднее всех векторов слов документа
    return np.mean(word2vec_model.wv[doc], axis=0)

# Преобразование каждого текста в усредненный вектор
daf['doc_vector'] = daf['preprocessed_text'].apply(lambda doc: document_vector(model, doc))

# Получение массива векторов и меток

x_vectors = np.array(daf['doc_vector'].tolist())
y_train = daf['category'].values

In [ ]:
x_vectors.shape

(265, 50)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import SVC

# Инициализация классификатора SVC
classifier = SVC()

# Создание скорера для f1-score
f1_scorer = make_scorer(f1_score)

# Выполнение кросс-валидации с вычислением f1-score
f1_scores = cross_val_score(classifier, x_vectors, y_train, cv=5, scoring=f1_scorer)

# Вывод результатов
print("Средний F1-счет: %0.2f (+/- %0.2f)" % (f1_scores.mean(), f1_scores.std() * 2))


Средний F1-счет: 0.64 (+/- 0.29)


In [ ]:
classifier.fit(x_vectors, y_train)

SVC()

In [ ]:
words = []

def get_vector(model, tokenized_text):
    return document_vector(model, tokenized_text)

# берем поступаемый текст и разделяем его на части: Speaker N, *реплика этого спикера*

for i in text:
    speaker, utterance = str(i).split(': ')
    utterance = utterance.strip() # '*text*'
    words.append(utterance)

dictionary = {'text': words}
test = pd.DataFrame(dictionary)

test['preprocesed_text'] = test.apply(lambda row: preprocess(row['text'], stop_words, punctuation_marks, morph), axis=1)

for i in range(len(text)):
    speaker, utterance = text[i].split(': ')
    speaker = speaker.strip() # 'Speaker A'
    utterance = utterance.strip() # '*text*'

    new_vector = get_vector(model, test['preprocesed_text'][i])
    new_vector = np.array([new_vector])

    prediction = classifier.predict(new_vector)

    if speaker not in speakers:
      speakers[speaker] = []

    if prediction:
      speakers[speaker].append(utterance)

In [ ]:
speakers

{'Говорящий 1': ['А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
  'Что мы тогда ставим, 21 или другие варианты есть?',
  'Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
  'Что, мы снова ставим 1?',
  'Ну, посмотрим. Будет смешно, если не поставить четыре.',
  'Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
  'Давайте 45 тогда, да?',
  'Давайте единицу.'],
 'Говорящий 2': ['Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
  'Так, я предлагаю поставить 7.',
  'Да, я ставлю 18, конечно, заканчиваем.',
  'А что нам делать-то нужно?',
  'Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',

### Загрузка модели и классификатора для предсказаний

In [ ]:
import pickle
import numpy as np
from gensim.models import Word2Vec

model = Word2Vec.load('/content/drive/MyDrive/Colab_Notebooks/models/word2vec_class/word2vec-classification-50')

with open('/content/drive/MyDrive/Colab_Notebooks/models/word2vec_class/classifier.pkl', 'rb') as file:
    classifier = pickle.load(file)

In [ ]:
speakers = dict()

In [ ]:
text = ['Говорящий 1: Всем привет. Мы, кажется, все собрались, да? У нас должно было быть четверо. Меня хорошо слышно? Хорошо. Я, простите, больше не буду без камеры. У нас сейчас важный такт. Нам нужно выбрать, кто из нас будет подавать заявки в форме. И я пришлю форму.',
        'Говорящий 2: Я буду делать.',
        'Говорящий 1: Отлично, давайте. Сейчас вам в чате зума. Все, получили? А сколько у нас минут заполнено?',
        'Говорящий 2: Пять минут.',
        'Говорящий 1: Ага. Тогда какие предложения по ставкам?',
        'Говорящий 2: Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
        'Говорящий 1: Ну да, мне тоже кажется, что или надо больше половины, чтобы сразу на свою сторону, или минимальную, чтобы другая команда ресурсы пожгла.',
        'Говорящий 2: Угу. Так. Проверочный код вашей команды. А где его проверочный код получить?',
        'Говорящий 1: А вот тоже я отправила сообщение тут же.',
        'Говорящий 1: А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
        'Говорящий 2: Ну 13, да, интересно в принципе ставка, если с учетом того, что она заканчивается на неудобную цифру. Так, а у нас играет кто получается?',
        'Говорящий 1: Мы наверное с четвертой играем.',
        'Говорящий 2: У нас осталось 1 минута. Так, давайте, кто за единицу голосует.',
        'Говорящий 1: Я за единицу. Ага.',
        'Говорящий 2: Я за единицу. И... Я за единицу.',
        'Говорящий 2: Так, отлично. Так, единица или 13?',
        'Говорящий 1: Галина, я вас плохо слышу, если честно.',
        'Говорящий 2: Так, а мы разговариваем, Милана, с вами, да, вдвоем?',
        'Говорящий 1: Я так понимаю, у Анны что-то с микрофоном? Юлия, вы с нами?',
        'Говорящий 2: Давайте, все, отправляю единицу. Ставку сделали. Нужно выйти, да, теперь?',
        'Говорящий 1: Она сама отправится.',
        'Говорящий 2: А у меня пришло приглашение выйти из зала.',
        'Говорящий 1: О, отлично. Спасибо, Юлия. Галина, вы с нами? Все, мы все теперь в чате. Хорошо.',
        'Говорящий 1: Что мы тогда ставим, 21 или другие варианты есть?',
        'Говорящий 1: Так, Юлия Анна пишет 21, Галина 29.',
        'Говорящий 1: Нет, друзья, у них максимум 20 ставка, нам же не нужно 29. У нас 21 это максимум.',
        'Говорящий 1: Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
        'Говорящий 1: Да, точно. У всех, видимо, что-то со звуком. Ничего, я буду озвучивать.',
        'Говорящий 2: Так, все здесь?',
        'Говорящий 1: Здесь.',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 1: У нас предложение поставить 10, потому что чуть больше, чем половина от их суммы. Если они поставят больше, то у них будет меньше 100% чем у нас.',
        'Говорящий 2: Ну, смотрите, а это же будет последний ход?',
        'Говорящий 1: Что?',
        'Говорящий 1: Это же будет последний раунд?',
        'Говорящий 1: Нет, мы играем пока не перетащим камень или пока не закончатся монеты.',
        'Говорящий 2: Но я предлагаю 7. Почему? Потому что даже если они поставят чуть больше 8, у них в остатке останется 11, а у нас тут чуть больше монет 21.',
        'Говорящий 1: Ну, здесь я просто не знаю, насколько велика вероятность, что они поставят меньше 7.',
        'Говорящий 2: Нет, ну тут меньше 7 они точно не поставят. Хотя, с другой стороны.',
        'Говорящий 1: Мы можем поставить чуть больше половины. Просто тогда будет вероятность больше, что мы...',
        'Говорящий 2: Если они перетянут, допустим, если они поставят 8, и  них в остатке останется 11, а у нас 21, то у нас будет преимущество на следующие шаги, у нас останется чуть больше монет. Просто если они сейчас поставят 8, мы поставим 10, и мы перетянем камень, у нас останется 18.',
        'Говорящий 1: Поздравляю, в одном шаге от победы.',
        'Говорящий 2: Так, я предлагаю поставить 7.',
        'Говорящий 1: Так мы можем все поставить, мы уже точно их выиграем, если поставим 18. У них меньше. Так что ставим 18 и все. Камень наш.',
        'Говорящий 2: А, ну да, логично. Так, а почему-то у меня не возвращается моя ставка.',
        'Говорящий 1: 18 у нас же, правильно?',
        'Говорящий 1: Супер.',
        'Говорящий 2: 18, правильно.',
        'Говорящий 1: Интересно, если бы мы поставили 15, мы бы вышли с монетами?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Ну, очень, как будто слабо. Сейчас слышно.',
        'Говорящий 1: Пока мы отдыхаем. И ждем следующий раунд.',
        'Говорящий 2: Я поняла.',
        'Говорящий 1: Можем, не знаю, подумать, что... Может быть, в следующий раунд, короче, на следующем ходе, можно зафиксировать, какие стратегии команды-победители используют, чтобы примерно знать, может быть, с кем может столкнуться в следующем раунде. Посмотреть пока будет демонстрировать.',
        'Говорящий 2: Да, согласна, что нужно изучить.',
        'Говорящий 1: Ну, я пока могу только сказать, что вроде восьмая прям осторожничает. Вторая в начале тоже не очень-то. А вот пятая вторым шагом, она прям огромную ставку сделала. Видимо, возможно, они так же будут делать в следующий раз. Знать бы еще, с кем мы будем играть. Ну, это она сделала, когда у нее был один шаг до победы. Просто даже в этот момент пытается безопасно сделать ход. Кто-то сразу готов.',
        'Говорящий 1: Да, мы таким же составом.',

        'Говорящий 1: Что, мы снова ставим 1?',
        'Говорящий 2: Меня слышно?',
        'Говорящий 1: Да, слышно.',
        'Говорящий 2: Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',
        'Говорящий 1: Мне кажется, надо меньше 10 тоже, я согласна. Это хорошо работает, когда они жгут ресурсы, и потом мы с ними работаем. Мне кажется, или 2, или... Я, конечно, за 2, чтобы... они осторожные. Но вторая, вначале у них была 23 ставка, то есть они настолько осторожные.',
        'Говорящий 2: Я не могу найти ссылку, куда ставить ставку.',
        'Говорящий 1: Сейчас скину.',

        'Говорящий 3: Предлагаю поставить полностью ставку. Сколько осталось. И мы их выиграем. Они не поставят полностью всю.',
        'Говорящий 1: О, она здорово у вас голос. Тебя слышно. Я поддерживаю, на самом деле.',
        'Говорящий 2: Да, абсолютно я согласна, что надо поставить максимум.',
        'Говорящий 3: Их надо скинуть 100%.',
        'Говорящий 1: Галина, ставка идеальна, как вы угадали.',
        'Говорящий 2: Вообще, я не знаю. Это какое-то подсознание. Я не знаю, даже как это.',
        'Говорящий 1: Класс.',
        'Говорящий 2: Так, 47. Все, 47 ставка, все отлично. 47 же, я же не ошибаюсь.',
        'Говорящий 1: Да.',
        'Говорящий 2: Так, отлично, все. Все, ставки сделаны.',

        'Говорящий 1: В финале будет интересно.',
        'Говорящий 2: Да, разворачиваются события, конечно.',
        'Говорящий 1: Но я всех поздравляю.',
        'Говорящий 2: Да, ну и точно. Я в какой-то момент подумала о том, что если они сейчас поставят 48 и все. Думаю, интересно, рискнут они или нет.',
        'Говорящий 1: Да, но тут как бы не рискнешь и не победишь, наверное. Четвертая рисковая, окей. Ну у меня зафиксировано, что они начали с 4. Правда, четвертая, по-моему.',
        'Говорящий 2: Да, и они вообще всю игру делают. Там 46, 42, у них все четные цифры. Я обратила просто внимание.',
        'Говорящий 1: Это хорошее наблюдение.',
        'Говорящий 1: Так, пятая 1, потом 11. Запишем.',

        'Говорящий 1: Три? Ну, что-нибудь такое, да, наверное.',
        'Говорящий 2: Я заметила, что если начинать примерно с мелких ставок, либо с 13, и потом ставить поменьше. Ну, короче, если начинать с 5 или 3, а потом десятые прибавлять, либо наоборот, сначала с десятых, там больше 10 ставить, например 11 или 13, а потом уменьшать единицы. Ну, то есть, тут надо тактику.',

        'Говорящий 1: Ну что, три, как договаривались? Пятые вроде начали в прошлый раз с 11.',
        'Говорящий 3: Да, давайте три, чтобы больше ресурсов осталось.',
        'Говорящий 2: Да-да-да, будем в стройке.',
        'Говорящий 1: Хорошо. 30 секунд это, конечно, жестко будет.',
        'Говорящий 2: Да, очень. Это вообще очень мало времени. Я успеваю только. Ну, жаль. У нас время почему-то не идет.',
        'Говорящий 1: Видимо, еще не все присоединились.',
        'Говорящий 2: Ну все, я уже тройку отправила. Ладно, согласна с вами. Надо выиграть.',
        'Говорящий 1: Чувствую ощущение, что в этом раунде будут больше рисковать.',
        'Говорящий 2: Угу.',
        'Говорящий 1: Ну, посмотрим. Будет смешно, если не поставить четыре.',
        'Говорящий 2: Это да.',

        'Говорящий 1: Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
        'Говорящий 3: Да, мне тоже кажется, что они сейчас пойдут в ва-банк, они полностью поставят, и они тогда выиграют.',
        'Говорящий 1: Тогда надо 45.',
        'Говорящий 3: У нас сколько осталось?',
        'Говорящий 2: У нас осталось 47.',
        'Говорящий 3: 47.',
        'Говорящий 2: Это же следовательно, что они будут рисковать 100%.',
        'Говорящий 3: Да.',
        'Говорящий 1: Давайте 45 тогда, да?',
        'Говорящий 3: Я думаю, да.',
        'Говорящий 1: И у нас останется на два хода перечень, если что.',
        'Говорящий 2: Ну да.',
        'Говорящий 1: Надеюсь, они не подумают, что мы решим так сделать. А то эти эффективные игры.',
        'Говорящий 3: 45.',
        'Говорящий 1: 45 вроде, да.',
        'Говорящий 2: У меня 45 и только ссылочка, пришлите еще раз кого-то.',
        'Говорящий 1: Сейчас.',
        'Говорящий 2: Я не успела ставку сделать, верните, пожалуйста.',

        'Говорящий 1: Из нас уже никто не выиграет.',
        'Говорящий 2: Я предлагаю ставить единицу и как бы оставить... Ну, то есть ставить единицу уже тогда.',
        'Говорящий 1: Ну, придётся, видимо. Потому что если двойку, то... Ну, всё равно... Ну, короче, уже на два хода ни у кого не останется.',
        'Говорящий 1: Давайте единицу.',

        'Говорящий 3: Ну, выбора у нас особого нет.',
        'Говорящий 1: Ну да. Интересный финал. Единица, так единица.',
        'Говорящий 3: Что у нас, камень остается внизу, то победитель или нет? Или я что-то путаю?',
        'Говорящий 1: Ну, он остается на второй ступени, а победителя нет, да.',
        'Говорящий 3: То есть камень упадет вниз, вниз со второй ступени, потому что одинаковая будет ставка.',
        'Говорящий 1: Ну, он останется на второй, но это не засчитывается как победа пятой команды, потому что до вершины его не добили.',
        'Говорящий 3: Ну, понятно. Спасибо. Мы молодцы, мы не даем выиграть.',
        'Говорящий 1: Ну, блин, это... Этот ход, когда 45, конечно. Интересный был.',
        'Говорящий 3: Давай, я согласна!'
]

In [ ]:
words = []

def get_vector(model, tokenized_text):
    return document_vector(model, tokenized_text)

# берем поступаемый текст и разделяем его на части: Speaker N, *реплика этого спикера*

for i in text:
    speaker, utterance = str(i).split(': ')
    utterance = utterance.strip() # '*text*'
    words.append(utterance)

dictionary = {'text': words}
test = pd.DataFrame(dictionary)

test['preprocesed_text'] = test.apply(lambda row: preprocess(row['text'], stop_words, punctuation_marks, morph), axis=1)

for i in range(len(text)):
    speaker, utterance = text[i].split(': ')
    speaker = speaker.strip() # 'Speaker A'
    utterance = utterance.strip() # '*text*'

    new_vector = get_vector(model, test['preprocesed_text'][i])
    new_vector = np.array([new_vector])

    prediction = classifier.predict(new_vector)

    if speaker not in speakers:
      speakers[speaker] = []

    if prediction:
      speakers[speaker].append(utterance)

In [ ]:
speakers

{'Говорящий 1': ['А, ну вы пишете 13, предлагаете. А что насчет 1, 2, что вы думаете по этому поводу?',
  'Что мы тогда ставим, 21 или другие варианты есть?',
  'Что, 21? Галина, поставите? Если они пойдут ва-банк, у них 20, а мы 21 поставим. Ну да, нужно точно. Галина, вы поставите? Спасибо. Супер.',
  'Что, мы снова ставим 1?',
  'Ну, посмотрим. Будет смешно, если не поставить четыре.',
  'Блин. Мне кажется, они пойдут в ва-банк, и нужно столько, чтобы перекрыть их.',
  'Давайте 45 тогда, да?',
  'Давайте единицу.'],
 'Говорящий 2': ['Я предлагаю начать с минимальных ставок. Для того, чтобы распределить своих оппонентов на более высокие ставки, предлагаю сделать с единички. Чтобы у нас осталось больше денег, у нас будет больше остатков перебить.',
  'Так, я предлагаю поставить 7.',
  'Ну и что хотите ставить? Я предлагаю поставить ну как бы либо 13, либо 17, потому что команда 2 всегда ставит 20, 22, такие.',
  'Я заметила, что если начинать примерно с мелких ставок, либо с 13, и пот